In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
tf.keras.utils.set_random_seed(123)

In [3]:
df = pd.read_csv("./src/2024_6_3_Data.txt", sep='\t')

In [4]:
df.head()

,Date,Command_door,Current (A),Unnamed: 3
0,3/6/2024 14:51:20.58,COMMAND_CLOSE,"0,567",NaN
1,3/6/2024 14:51:20.137,COMMAND_CLOSE,"0,548",NaN
2,3/6/2024 14:51:20.313,COMMAND_CLOSE,"0,547",NaN
3,3/6/2024 14:51:20.406,COMMAND_CLOSE,"0,551",NaN
4,3/6/2024 14:51:20.483,COMMAND_CLOSE,"0,56",NaN


In [5]:
df.columns

Index(['Date ', 'Command_door ', 'Current (A) ', 'Unnamed: 3'], dtype='object')

In [6]:
df['Current (A) '] = df['Current (A) '].str.replace(',','.').astype(float)

In [7]:
df['Obs Number'] = 0

In [8]:
df = df.drop('Unnamed: 3', axis=1)

In [9]:
df.head()

,Date,Command_door,Current (A),Obs Number
0,3/6/2024 14:51:20.58,COMMAND_CLOSE,0.567,0
1,3/6/2024 14:51:20.137,COMMAND_CLOSE,0.548,0
2,3/6/2024 14:51:20.313,COMMAND_CLOSE,0.547,0
3,3/6/2024 14:51:20.406,COMMAND_CLOSE,0.551,0
4,3/6/2024 14:51:20.483,COMMAND_CLOSE,0.560,0


In [10]:
obs_number = 0

In [11]:
len(df)

2193

In [12]:
for i in range(1,len(df)):
    if df.loc[i,'Command_door '] != df.loc[i-1,'Command_door ']:
        obs_number += 1
    df.loc[i,'Obs Number'] = obs_number        
    

In [13]:
# Valores sin repeticiones en la columna
df['Obs Number'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int64)

In [14]:
df['Command_door '].unique()

array(['COMMAND_CLOSE', 'COMMAND_OPEN'], dtype=object)

In [15]:
close = df[df['Command_door '] == 'COMMAND_CLOSE' ]

In [16]:
close.head()

,Date,Command_door,Current (A),Obs Number
0,3/6/2024 14:51:20.58,COMMAND_CLOSE,0.567,0
1,3/6/2024 14:51:20.137,COMMAND_CLOSE,0.548,0
2,3/6/2024 14:51:20.313,COMMAND_CLOSE,0.547,0
3,3/6/2024 14:51:20.406,COMMAND_CLOSE,0.551,0
4,3/6/2024 14:51:20.483,COMMAND_CLOSE,0.560,0


In [17]:
data = close.pivot_table(values = 'Current (A) ', aggfunc=['mean', 'median','std', 'min','max'], index='Obs Number')

In [18]:
data.head()

,mean,median,std,min,max
,Current (A),Current (A),Current (A),Current (A),Current (A)
Obs Number,,,,,
0,0.547672,0.548,0.010250,0.506,0.569
2,0.566139,0.358,0.603041,-0.697,1.819
4,0.559519,0.485,0.645010,-1.122,1.819
6,0.474645,0.286,0.638256,-0.904,1.814
8,0.446937,0.304,0.689106,-1.390,1.848


In [19]:
X = pd.concat([data.head(), data.tail()])

In [20]:
X

,mean,median,std,min,max
,Current (A),Current (A),Current (A),Current (A),Current (A)
Obs Number,,,,,
0,0.547672,0.5480,0.010250,0.506,0.569
2,0.566139,0.3580,0.603041,-0.697,1.819
4,0.559519,0.4850,0.645010,-1.122,1.819
6,0.474645,0.2860,0.638256,-0.904,1.814
8,0.446937,0.3040,0.689106,-1.390,1.848
20,1.064522,1.1975,0.771661,-1.186,1.873
22,1.175400,1.3675,0.649127,-0.804,1.867
24,1.098318,1.3160,0.748465,-0.706,1.883


In [21]:
y = np.array([0,0,0,0,0,1,1,1,1,1])

# Red neuronal sencilla

In [22]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.InputLayer(input_shape=(X.shape[1],)),
     tf.keras.layers.Dense(3),
     tf.keras.layers.Dense(2, activation='softmax')   
    ]
)

In [23]:
model.compile(loss='mse', metrics=['mae'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 18        
                                                                 
 dense_1 (Dense)             (None, 2)                 8         
                                                                 
Total params: 26 (104.00 Byte)
Trainable params: 26 (104.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
X = X.values.reshape(X.shape[0], X.shape[1], 1)

In [26]:
model.fit(X,y, epochs=100)

Epoch 1/100


1/1 [==============================] - 1s 837ms/step - loss: 0.4631 - mae: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 13ms/step - loss: 0.4617 - mae: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 16ms/step - loss: 0.4606 - mae: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 16ms/step - loss: 0.4596 - mae: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 12ms/step - loss: 0.4587 - mae: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 12ms/step - loss: 0.4578 - mae: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 11ms/step - loss: 0.4569 - mae: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 12ms/step - loss: 0.4561 - mae: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 12ms/step - loss: 0.4553 - mae: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 13ms/step - loss: 0.4544 - mae: 0.5000
Epoch 11/100
1/1 [==============================] - 0s 14ms/step -

In [27]:
loss, accuraccy = model.evaluate(X,y)

1/1 [==============================] - 0s 212ms/step - loss: 0.3009 - mae: 0.5000


In [28]:
loss

0.300913006067276

In [29]:
accuraccy

0.5

In [30]:
y_pred_proba = model.predict(X)

1/1 [==============================] - 0s 105ms/step


In [31]:
y_pred = y_pred_proba.argmax(axis=1)

In [32]:
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [33]:
y

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])